In [28]:
import numpy as np

In [1]:
def read_input(filename, split_lines = True, convert_to_int = True):
    f = open(filename)
    raw = f.read()[:-1]
    f.close()
    data = raw
    if split_lines:
        data = data.split('\n')
    if convert_to_int:
        data = [int(item) for item in data]
    return data

In [3]:
sample_data, data = read_input('sample.txt', False, False), read_input('input.txt', False, False)

In [256]:
class BingoBoard:
    def __init__(self, board_string, board_id=None):
        self.cell_values = np.zeros((5,5))
        for i, row in enumerate(board_string.split('\n')):
            self.cell_values[i] = row.split()
        self.cells_drawn = np.zeros((5,5))
        self.board_id = board_id
        self.log = []
    
    def draw_number(self, number):
        if number in self.cell_values:
            self.cells_drawn[np.where(self.cell_values == number)] = 1
        self.log.append(number)
    
    def check_bingo(self):
        if sum([sum(row) == 5 for row in self.cells_drawn]) > 0:
            return True
        elif sum([sum(row) == 5 for row in self.cells_drawn.transpose()]) > 0:
            return True
        else:
            return False
    
    def calc_winning_score(self):
        unmarked_sum = 0
        for i in range(5):
            for j in range(5):
                if self.cells_drawn[i,j] == 0:
                    unmarked_sum += self.cell_values[i,j]
        return unmarked_sum * self.log[-1]

        
class BingoGame:
    def __init__(self, generated_order):
        self.boards = []
        self.draws, raw_boards = self.split_input(generated_order)
        for i, board in enumerate(raw_boards):
            self.boards.append(BingoBoard(board, i))
        del raw_boards
        self.winner_id = None
        self.winning_order = []
    
    @staticmethod
    def split_input(generated_order):
        cut = generated_order.split('\n\n')
        return cut[0].split(','), cut[1:]
    
    def play(self):
        for number in self.draws:
            for board in self.boards:
                board.draw_number(int(number))
                if board.check_bingo():
                    self.winner_id = board.board_id
                    break
            if self.winner_id != None:
                break
        return self.boards[self.winner_id].calc_winning_score()
    
    def lose(self):
        while len(self.winning_order) < len(self.boards):
            number = self.draws.pop(0)
            for board in self.boards:
                if board.board_id in self.winning_order:
                    continue
                board.draw_number(int(number))
                if board.check_bingo():
                    self.winning_order.append(board.board_id)
        return self.boards[self.winning_order[-1]].calc_winning_score()

In [257]:
test1 = BingoGame(sample_data)

In [258]:
test1.play() == 4512

True

In [259]:
%%time
BingoGame(data).play()

CPU times: user 144 ms, sys: 3.73 ms, total: 148 ms
Wall time: 146 ms


23177.0

In [260]:
test2 = BingoGame(sample_data)

In [261]:
test2.lose() == 1924

True

In [262]:
%%time
BingoGame(data).lose()

CPU times: user 424 ms, sys: 14 ms, total: 438 ms
Wall time: 441 ms


6804.0